# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [1]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [2]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=200_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

(200000, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

#### Análise de Target

In [3]:
df['loan_status'].value_counts(dropna=False, normalize=True) * 100

loan_status
Fully Paid            70.4960
Charged Off           17.5450
Current               11.3185
Late (31-120 days)     0.3925
In Grace Period        0.1735
Late (16-30 days)      0.0740
Default                0.0005
Name: proportion, dtype: float64

In [4]:
TARGET = 'target_risco_credito'

status_de_risco = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    #"Late (16-30 days)"
]
df[TARGET] = df["loan_status"].isin(status_de_risco).astype(int)

df[TARGET].value_counts(dropna=False, normalize=True) * 100

target_risco_credito
0    82.062
1    17.938
Name: proportion, dtype: float64

### Safras

In [5]:
date_col=['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d','next_pymnt_d']

for col in date_col:

    df[col] = pd.to_datetime(df[col], format="%b-%Y")


df["safra"] = df["issue_d"].dt.to_period("M").astype(str)  # formato 'YYYY-MM'

print(df["safra"].min())
print(df["safra"].max())

2015-07
2015-12


### Captura Inicial

In [6]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    target_col=TARGET,
    default_keys=["id"]
)

trail.take_snapshot(df, name="original")

In [7]:
trail.describe_snapshot("original")


📄 Descrição do snapshot 'original':

▶️ Shape: (200000, 153)
▶️ Chaves de duplicação: ['id']
   • Duplicatas nas chaves: 0

🧱 Tipos de dados:


id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
funded_amnt_inv          float64
                          ...   
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
target_risco_credito       int32
safra                     object
Length: 153, dtype: object


🔎 Colunas detectadas automaticamente:
   • Numéricas (88): ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_withi

member_id                                     200000
revol_bal_joint                               200000
sec_app_fico_range_high                       200000
sec_app_earliest_cr_line                      200000
sec_app_inq_last_6mths                        200000
sec_app_mort_acc                              200000
sec_app_open_acc                              200000
sec_app_revol_util                            200000
sec_app_open_act_il                           200000
sec_app_num_rev_accts                         200000
sec_app_chargeoff_within_12_mths              200000
sec_app_collections_12_mths_ex_med            200000
sec_app_mths_since_last_major_derog           200000
sec_app_fico_range_low                        200000
desc                                          199989
dti_joint                                     199491
annual_inc_joint                              199489
verification_status_joint                     199489
orig_projected_additional_accrued_interest    


📊 Estatísticas numéricas:


,count,mean,std,min,25%,50%,75%,max
id,200000.0,6.229632e+07,3.941141e+06,56705.00,59411728.75,62217530.00,65644567.00,68617057.00
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,200000.0,1.527816e+04,8.651134e+03,1000.00,8500.00,14000.00,20000.00,35000.00
funded_amnt,200000.0,1.527816e+04,8.651134e+03,1000.00,8500.00,14000.00,20000.00,35000.00
funded_amnt_inv,200000.0,1.526951e+04,8.646320e+03,900.00,8475.00,14000.00,20000.00,35000.00
...,...,...,...,...,...,...,...,...
hardship_last_payment_amount,1409.0,1.860614e+02,1.812045e+02,0.02,44.95,134.56,275.29,927.79
settlement_amount,5840.0,5.015406e+03,3.557532e+03,130.00,2192.49,4344.99,7000.00,26242.50
settlement_percentage,5840.0,4.710027e+01,5.753855e+00,20.00,45.00,45.00,50.00,97.66
settlement_term,5840.0,1.348545e+01,7.537550e+00,0.00,8.00,13.50,18.00,65.00



🏷️ Estatísticas categóricas:


,count,unique,top,freq
term,200000,2,36 months,134643
grade,200000,7,B,59333
sub_grade,200000,35,C1,12803
emp_title,187870,66136,Teacher,3823
emp_length,187897,11,10+ years,67516
home_ownership,200000,4,MORTGAGE,99618
verification_status,200000,3,Source Verified,84756
loan_status,200000,7,Fully Paid,140992
pymnt_plan,200000,2,n,199978
url,200000,200000,https://lendingclub.com/browse/loanDetail.acti...,1



📈 Histogramas (categorias apenas):
  term: 2 valores distintos (top 3: {' 36 months': 134643, ' 60 months': 65357})
  grade: 7 valores distintos (top 3: {'B': 59333, 'C': 57429, 'A': 34866})
  sub_grade: 35 valores distintos (top 3: {'C1': 12803, 'B3': 12497, 'B4': 12214})
  emp_length: 12 valores distintos (top 3: {'10+ years': 67516, '2 years': 17529, '< 1 year': 16736})
  home_ownership: 4 valores distintos (top 3: {'MORTGAGE': 99618, 'RENT': 77978, 'OWN': 22403})
  verification_status: 3 valores distintos (top 3: {'Source Verified': 84756, 'Verified': 60961, 'Not Verified': 54283})
  loan_status: 7 valores distintos (top 3: {'Fully Paid': 140992, 'Charged Off': 35090, 'Current': 22637})
  pymnt_plan: 2 valores distintos (top 3: {'n': 199978, 'y': 22})
  purpose: 13 valores distintos (top 3: {'debt_consolidation': 114750, 'credit_card': 50817, 'home_improvement': 12599})
  title: 15 valores distintos (top 3: {'Debt consolidation': 114692, 'Credit card refinancing': 50811, 'Home imp

#### Limpeza com Vassoura

In [8]:
df[TARGET].dtypes

dtype('int32')

In [9]:
vs = Vassoura(
    df,
    target_col=TARGET,
    verbose='basic',
    engine='polars',
    adaptive_sampling=True,
    heuristics=[
        "graph_cut",
        "corr",
        "vif",
        "iv",
    ],
    thresholds={
        "missing": 0.60,
        "corr": 0.85,
        "vif": 5,
        "iv": 0.01,
        "graph_cut": 0.9,
    },
    n_steps=2,         # controla quantas remoções por iteração
    vif_n_steps=2      # VIF será sempre 1-step (a menos que você altere)
)

df_limpo = vs.run(recompute=True)

[Vassoura] Missing heuristic (thr>0.6)
  → dropped ['member_id', 'desc', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_len

INFO | Tipos detectados: 60 numéricas, 16 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 60 variáveis numéricas (engine=pandas)


  → dropped ['last_fico_range_low'] (corr>0.85)
  → dropped ['tot_hi_cred_lim'] (corr>0.85)
  → dropped ['bc_util'] (corr>0.85)


INFO | Tipos detectados: 57 numéricas, 16 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 57 variáveis numéricas (engine=pandas)


  → dropped ['total_bc_limit'] (corr>0.85)


...

INFO | Tipos detectados: 31 numéricas, 16 categóricas
INFO | Descartando 112598 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 31 variáveis


  → dropped ['dti'] (vif>5)


INFO | Tipos detectados: 30 numéricas, 16 categóricas
INFO | Descartando 112597 linha(s) com NaN/inf para cálculo de VIF
INFO | VIF calculado para 30 variáveis


[Vassoura] IV heuristic (thr<0.01) – removendo low IV
  → dropped ['emp_length', 'issue_d', 'pymnt_plan', 'delinq_2yrs', 'mths_since_last_delinq', 'initial_list_status', 'collections_12_mths_ex_med', 'application_type', 'tot_coll_amt', 'chargeoff_within_12_mths', 'delinq_amnt', 'num_accts_ever_120_pd', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'pub_rec_bankruptcies', 'tax_liens', 'total_il_high_credit_limit', 'hardship_flag', 'disbursement_method', 'safra'] (iv<0.01)


INFO | Tipos detectados: 17 numéricas, 10 categóricas
INFO | Método de correlação sugerido: spearman
INFO | Método spearman não suportado por engine 'polars'; utilizando pandas.
INFO | Matriz de correlação spearman calculada para 17 variáveis numéricas (engine=pandas)


In [10]:
df_limpo.head()

,term,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,loan_status,url,purpose,title,zip_code,addr_state,earliest_cr_line,inq_last_6mths,revol_bal,out_prncp_inv,total_rec_int,total_rec_late_fee,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,avg_cur_bal,bc_open_to_buy,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_tl_op_past_12m,percent_bc_gt_75,debt_settlement_flag,target_risco_credito
0,36 months,C,C4,leadman,MORTGAGE,55000.0,Not Verified,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,190xx,PA,2003-08-01,1.0,2765.0,0.00,821.72,0.0,0.0,2019-01-01,122.67,2019-03-01,20701.0,1506.0,3.0,3.0,1.0,4.0,4.0,3.0,0.0,N,0
1,36 months,C,C1,Engineer,MORTGAGE,65000.0,Not Verified,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,small_business,Business,577xx,SD,1999-12-01,4.0,21470.0,0.00,979.66,0.0,0.0,2016-06-01,926.35,2019-03-01,9733.0,57830.0,2.0,2.0,4.0,2.0,0.0,2.0,7.7,N,0
2,60 months,B,B4,truck driver,MORTGAGE,63000.0,Not Verified,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,NaN,605xx,IL,2000-08-01,0.0,7869.0,0.00,2705.92,0.0,0.0,2017-06-01,15813.30,2019-03-01,31617.0,2737.0,14.0,14.0,5.0,101.0,10.0,0.0,50.0,N,0
3,60 months,C,C5,Information Systems Officer,MORTGAGE,110000.0,Source Verified,Current,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,076xx,NJ,2008-09-01,0.0,7802.0,15897.65,12361.66,0.0,0.0,2019-02-01,829.90,2019-03-01,23192.0,54962.0,2.0,2.0,1.0,2.0,NaN,1.0,0.0,N,0
4,60 months,F,F1,Contract Specialist,MORTGAGE,104433.0,Source Verified,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,174xx,PA,1998-06-01,3.0,21929.0,0.00,1340.50,0.0,0.0,2016-07-01,10128.96,2018-03-01,27644.0,4567.0,4.0,4.0,6.0,4.0,1.0,4.0,60.0,N,0


In [11]:
vs.generate_report("vassoura_report_2.html")

INFO | Tipos detectados: 115 numéricas, 28 categóricas
INFO | Relatório gerado em vassoura_report_2.html


'vassoura_report_2.html'

#### Comparação após limpeza

In [ ]:
trail.take_snapshot(df_limpo, "limpo")

In [ ]:
trail.compare_snapshots("original", "limpo")

In [ ]:


print(len(loans_accepted))
loans_accepted.dropna(axis=0, inplace=True)
print(len(loans_accepted))

In [ ]:
loans_accepted.term.value_counts(dropna=False, normalize=True)*100

In [ ]:
for col in df.columns:
    tipos = df[col].apply(type).value_counts()
    if len(tipos) > 1:
        print(f"{col}: {tipos}")

In [ ]:
vsess = Vassoura(
    df,
    target_col='ever90m12',
    heuristics=['corr', 'vif'],
    thresholds={'corr': 0.9, 'vif': 10},
)
df_clean = vsess.run()
df_clean.head()

In [ ]:
# Gera relatório resumido
vsess.generate_report('example_report.html')